### In-Game Purchase Data Analysis

#### Background 

In this homework, you are ask to deal with a task of analyzing an "in-game purchase" dataset. In this game, players are able to to purchase optional items that enhance their playing experience. Now your task is to generate a report that breaks down the game's purchasing data into meaningful insights.

-----

#### Observable Trends

* There are 1163 active players. The vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.79%) with secondary groups falling between 15-19 (18.58%) and 25-29 (13.37%). 

* The age group that spends the most money is the 20-24 with 1,114.06 dollars as total purchase value and an average purchase of 4.32. In contrast, the demographic group that has the highest average purchase is the 35-39 with 4.76 and a total purchase value of 147.67. 
-----


In [1]:
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "./data/purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
# 單獨取出玩家名字進行處理
players_data = purchase_data.loc[:,["SN","Gender"]]
players_data = players_data.drop_duplicates()
pd.DataFrame([len(players_data)], columns=["Total Players"])

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.
* Create a summary data frame to hold the results
* Display the summary data frame

In [3]:
# 單獨取出物件名字進行處理
total_items = purchase_data.loc[:,["Item ID","Price"]]

# 所有消費紀錄、總金額及平均價格
total_purchase_num = len(total_items)
total_purchase_price = total_items["Price"].sum()
total_items_price = round(total_items["Price"].mean(),2)

# 所有商品數
total_items = total_items.drop_duplicates()
total_items_num = len(total_items)

item_array = [total_items_num, total_items_price, total_purchase_num, total_purchase_price]
item_df = pd.DataFrame(item_array, index=["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"],).T

item_df["Number of Unique Items"] = item_df["Number of Unique Items"].astype("int")
item_df["Number of Purchases"] = item_df["Number of Purchases"].astype("int")
item_df["Average Price"] = "$" + item_df["Average Price"].astype("string") 
item_df["Total Revenue"] = "$" + item_df["Total Revenue"].astype("string")
item_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# 用groupby統計性別人數及百分比
gender_num = players_data.groupby("Gender").size()
gender_percent = gender_num.transform(lambda x: round(x/len(players_data)*100,2))
gender_array  = [gender_percent.values, gender_num.values]

gender_df = pd.DataFrame(gender_array, index=["Percentage of Players", "Total Count"], 
                         columns=["Female", "Male", "Other / Non-Disclosed"]).T
gender_df["Total Count"] = gender_df["Total Count"].astype("int")
gender_df.sort_values(by="Total Count", ascending=False)

,Percentage of Players,Total Count
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain below values by gender
  * Purchase Count
  * Average Purchase Price  
  * Total Purchase Value
  * Average Purchase Total per Person by Gender
* Create a summary data frame to hold the results
* Display the summary data frame

In [5]:
players_data = purchase_data.loc[:,["Gender", "Price"]]

# 計算Purchase Count,Average Purchase Price,Total Purchase Value
gender_purchase = players_data.groupby("Gender").count()
gender_price_mean = players_data.groupby("Gender").mean()
gender_price_sum = players_data.groupby("Gender").sum()
gender_data = pd.concat([gender_purchase, gender_price_mean, gender_price_sum], axis=1)
gender_data.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]
gender_data["Average Purchase Price"] = round(gender_data["Average Purchase Price"].astype("float"),5)

# 計算Average Purchase Total per Person
per_price_array = []
for idx in range(len(gender_price_sum.values)):
    mean = gender_price_sum.values[idx][0]/gender_num[idx]
    per_price_array.append(mean.round(2))

# 新增Average Purchase Total per Person
gender_data["Average Purchase Total per Person"] = per_price_array
gender_data["Average Purchase Total per Person"] = "$" + gender_data["Average Purchase Total per Person"].astype("string")
gender_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Gender,,,,
Female,113,3.20301,361.94,$4.47
Male,652,3.01785,1967.64,$4.07
Other / Non-Disclosed,15,3.34600,50.19,$4.56


## Age Demographics

* Categorize players using the age groups. Hint: use pd.cut()
  * each group is for 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* round the percentage column to two decimal points
* Display Age Demographics Table

In [6]:
players_data = purchase_data.loc[:,["SN", "Age"]]
players_data = players_data.drop_duplicates()

# 利用pd.cut分區間
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
players_age = pd.cut(players_data["Age"], bins)

# 利用groupby取值，用transform計算百分比
age_df = pd.DataFrame(players_age)
age_gp = age_df.groupby("Age").size()
age_percent = age_gp.transform(lambda x: round(x/len(players_data)*100,2))

# 將值都放進array並一起轉成dataframe
age_array  = [age_percent.values, age_gp.values]
age_df = pd.DataFrame(age_array, index=["Percentage of Players", "Total Count"], 
                         columns=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]).T
age_df["Total Count"] = age_df["Total Count"].astype("int")
age_df

,Percentage of Players,Total Count
<10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40+,2.08,12


## Purchasing Analysis (Age)

* The below each broken into groups of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group
* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Display the summary data frame

In [7]:
players_data = purchase_data.loc[:,["Age", "Price"]]

# 利用pd.cut分區間
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
players_data["Age"] =pd.cut(players_data["Age"], bins, labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"])
age_df = pd.DataFrame(players_data)

# 計算Purchase Count,Average Purchase Price,Total Purchase Value
age_purchase = age_df.groupby("Age").count()
age_price_mean = age_df.groupby("Age").mean()
age_price_sum = age_df.groupby("Age").sum()
age_data = pd.concat([age_purchase, age_price_mean, age_price_sum], axis=1)
age_data.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]

# 計算Average Purchase Total per Person
per_price_array = []
for idx in range(len(age_price_sum.values)):
    mean = age_price_sum.values[idx][0]/age_gp[idx]
    per_price_array.append(mean.round(2))
    
# 新增Average Purchase Total per Person
age_data["Average Purchase Total per Person"] = per_price_array
age_data["Average Purchase Price"] = age_data["Average Purchase Price"].map(lambda x: "$%.2f" %round(x,2))
age_data["Total Purchase Value"] = age_data["Total Purchase Value"].map(lambda x: "$%.2f" %round(x,2))
age_data["Average Purchase Total per Person"] = age_data["Average Purchase Total per Person"].map(lambda x: "$%.2f" %round(x,2))
age_data.index.name= None
age_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Identify the the top 5 spenders in the game by total purchase value (top-5 players with highest total purchase value), then list (in a table):
  * SN(screen name)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
* Run basic calculations to obtain the results in the table below
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Display a preview of the summary data frame

In [8]:
players_data = purchase_data.loc[:,["SN", "Price"]]

# 計算所有玩家Purchase Count、Average Purchase Price、Total Purchase Value
players_purchase = players_data.groupby("SN").count()
players_price_mean = players_data.groupby("SN").mean()
players_price_sum = players_data.groupby("SN").sum()
players_data = pd.concat([players_purchase, players_price_mean, players_price_sum], axis=1)
players_data.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]

# 只取前五名的玩家
top_players = players_data.sort_values(by="Total Purchase Value", ascending=False).head(5)
top_players["Average Purchase Price"] = top_players["Average Purchase Price"].map(lambda x: "$%.2f" %round(x,2))
top_players["Total Purchase Value"] = top_players["Total Purchase Value"].map(lambda x: "$%.2f" %round(x,2))
top_players

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Top 5 most popular items by purchase count:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Retrieve the Item ID, Item Name, and Item Price columns
* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
* Create a summary data frame to hold the results
* Sort the purchase count column in descending order
* Display a preview of the summary data frame

In [9]:
players_data = purchase_data.loc[:,["Item ID", "Item Name", "Price"]]

# 計算所有商品的Purchase Count、Price、Total Purchase Value
items_purchase = players_data.groupby(["Item ID", "Item Name"]).count()
items_price_mean = players_data.groupby(["Item ID", "Item Name"]).mean()
items_price_sum = players_data.groupby(["Item ID", "Item Name"]).sum()
items_data = pd.concat([items_purchase, items_price_mean, items_price_sum], axis=1)
items_data.columns = ["Purchase Count", "Items Price", "Total Purchase Value"]


# 只取賣出數量前五名的商品
top_items = items_data.sort_values(by="Purchase Count", ascending=False).head(5)
top_items["Items Price"] = top_items["Items Price"].map(lambda x: "$%.2f" %round(x,2))
top_items["Total Purchase Value"] = top_items["Total Purchase Value"].map(lambda x: "$%.2f" %round(x,2))
top_items

,,Purchase Count,Items Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Top 5 most profitable items by total purchase value:
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
* Sort the above table by total purchase value in descending order
* Display a preview of the data frame

In [10]:
# 只取賣出總金額前五名的商品
top_items = items_data.sort_values(by="Total Purchase Value", ascending=False).head(5)
top_items["Items Price"] = top_items["Items Price"].map(lambda x: "$%.2f" %round(x,2))
top_items["Total Purchase Value"] = top_items["Total Purchase Value"].map(lambda x: "$%.2f" %round(x,2))
top_items

,,Purchase Count,Items Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
